In [3]:
import zipfile
import os

zip_path = '/content/Mushrooms(1).zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

print("✅ Extracted. Check the folder 'Mushrooms/' now.")



✅ Extracted. Check the folder 'Mushrooms/' now.


In [4]:

from PIL import Image, UnidentifiedImageError
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


root_dir = '/content/Mushrooms'
corrupted_files = []

for subdir, _, files in os.walk(root_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        try:
            with Image.open(file_path) as img:
                img.verify()
        except (UnidentifiedImageError, IOError, OSError):
            print(f"❌ Corrupted: {file_path}")
            corrupted_files.append(file_path)

for path in corrupted_files:
    os.remove(path)

print(f"✅ Removed {len(corrupted_files)} corrupted files.")


❌ Corrupted: Mushrooms/.DS_Store
✅ Removed 1 corrupted files.


In [5]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 9


In [6]:

def load_data(train_dir):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )
    val_generator = datagen.flow_from_directory(
        train_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )


    with open("class_indices.pkl", "wb") as f:
        pickle.dump(train_generator.class_indices, f)

    print("✅ Classes detected:", train_generator.class_indices)
    return train_generator, val_generator


In [7]:

def build_model():
    model = models.Sequential([
        layers.Input(shape=(*IMAGE_SIZE, 3)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [9]:

train_dir = 'Mushrooms'
train_gen, val_gen = load_data(train_dir)

model = build_model()
model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS)

model.save('softmax_model.keras')
print("✅ Model saved as softmax_model.pkl")


Found 5375 images belonging to 9 classes.
Found 1339 images belonging to 9 classes.
✅ Classes detected: {'Agaricus': 0, 'Amanita': 1, 'Boletus': 2, 'Cortinarius': 3, 'Entoloma': 4, 'Hygrocybe': 5, 'Lactarius': 6, 'Russula': 7, 'Suillus': 8}
Epoch 1/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 33s 179ms/step - accuracy: 0.2175 - loss: 2.0475 - val_accuracy: 0.3152 - val_loss: 1.8873
Epoch 2/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 28s 165ms/step - accuracy: 0.3407 - loss: 1.7933 - val_accuracy: 0.3376 - val_loss: 1.8641
Epoch 3/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 29s 169ms/step - accuracy: 0.4235 - loss: 1.6099 - val_accuracy: 0.3936 - val_loss: 1.7712
Epoch 4/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 28s 166ms/step - accuracy: 0.5258 - loss: 1.3626 - val_accuracy: 0.4287 - val_loss: 1.6933
Epoch 5/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 28s 165ms/step - accuracy: 0.6237 - loss: 1.0747 - val_accuracy: 0.3570 - val_loss: 2.0097
Epoch 6/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 28s 164ms/step - accuracy: 0.7472 - loss: 0.7369 - val_accuracy: